## 1. 감성 분석


감성 분석은 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성 수치를 계산하는 방법을 이용합니다.
<br>
감성 지수는 긍정 감성 지수와 부정 감성 지수로 구성되며 이들 지수를 합산해 긍정 감성 또는 부정 감성을 결정합니다.
<br>
이러한 감성 분석은 지도학습과 비지도 학습 방식으로 나누어집니다.

1-1 지도학습

In [1]:
import pandas as pd

review_df = pd.read_csv('labeledTrainData.tsv', header=0, sep="\t", quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


1-1-1 re 모듈에 있는 정규표현식을 이용하여 아래의 코드를 채워주세요.

In [8]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

# 파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x))

1-1-2 결정값 클래스인 sentiment 칼럼을 별도로 추출해 결정값 데이트를 나누고, 원본 데이터 세트에서 id 와 setiment 칼럼을 삭제해 피쳐 데이터를 생성해주세요.

In [9]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id', 'sentiment'], axis = 1, inplace = False)
X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

1-1-3 주석을 참고하여, Count벡터화를 적용하여 pipeline 객체를 만드는 코드를 작성하고 학습 예측 수행해주세요.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행.
# LogisticRegression의 C는 10으로 설정.
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words = 'english', ngram_range = (1,2))),
    ('lr_clf', LogisticRegression(solver = 'liblinear', C = 10))])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8859, ROC-AUC는 0.9503


1-1-4 주석을 참고하여, TF -IDF벡터화를 적용하여 pipeline 객체를 만드는 코드를 작성하고 학습 예측 수행해주세요.

In [13]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행.
# LogisticRegression의 C는 10으로 설정.
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english', ngram_range = (1,2))),
    ('lr_clf', LogisticRegression(solver = 'liblinear', C = 10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8936, ROC-AUC는 0.9598


1-2 비지도학습 : SentiWordNet을 이용한 감성분석

In [14]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basque_grammars.zip.
[nltk_data]   

KeyboardInterrupt: 

아래의 코드를 실행해주세요

In [ ]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return


1-2-1 주석을 기반으로 코드를 채워주세요.

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0

    lemmatizer = #채워주세요
    raw_sentences = #채워주세요
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출
        tagged_sentence = #채워주세요
        for word , tag in tagged_sentence:

            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성.
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산.
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += #채워주세요
            tokens_count += 1

    if not tokens_count:
        return 0

    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1

    return 0

아래의 코드를 실행하여 SentiWordNet 의 감성분석 예측 성능을 살펴봐주세요.

In [ ]:
review_df['preds'] = review_df['review'].apply( lambda x : swn_polarity(x) )
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix( y_target, preds))
print("정확도:", np.round(accuracy_score(y_target , preds), 4))
print("정밀도:", np.round(precision_score(y_target , preds),4))
print("재현율:", np.round(recall_score(y_target, preds), 4))

1-3 비지도학습 : VADER을 이용한 감성 분석

1-3-1 VADER을 이용하여 아래의 코드를 채워주세요.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer =#채워주세요
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

1-3-2 compound 값에 기반하여 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환하는 함수를 만들기 위해 아래의 코드를 채워주세요.

In [ ]:
def vader_polarity(review,threshold=0.1):
    analyzer = #채워주세요
    scores = #채워주세요
    # compound 값에 기반하여 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
    agg_score =#채워주세요
    final_sentiment =#채워주세요
    return final_sentiment

# apply lambda 식을 이용하여 레코드별로 vader_polarity( )를 수행하고 결과를 'vader_preds'에 저장
review_df['vader_preds'] = review_df['review'].apply( lambda x : vader_polarity(x, 0.1) )
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

print(confusion_matrix( y_target, vader_preds))
print("정확도:", np.round(accuracy_score(y_target , vader_preds),4))
print("정밀도:", np.round(precision_score(y_target , vader_preds),4))
print("재현율:", np.round(recall_score(y_target, vader_preds),4))

1-3-3 정확도, 정밀도, 재현율을 고려했을 때, SentiWordNet 과 VADER중 어떤것이 더 이 데이터셋에서 성능이 좋다고 할수 있나요?

답:

## 2. 문서 군집화

 Tripadvisor(호텔), Edmunds.com(자동차), Amazon.com(전자제품) 사이트에서 가져온 리뷰 문서를 활용하여,<br>비슷한 텍스트 구성의 문서를 군집화를 해보는 문제입니다.

파일명과 파일내용을 칼럼으로 가지는 DataFrame을 만드는 코드입니다.<br>
실행하여 DataFrame인 document_df이 어떻게 구성되어 있는지 확인해주세요

In [ ]:
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 700)

# 'topics' 파일 압축을 풀고 'topics' 파일의 위치를 넣어 실행해주세요
path = r""
# path로 지정한 디렉터리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합,
# 개별 파일들의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')

    # 절대경로로 주어진 file 명을 가공. 만일 Linux에서 수행시에는 아래 \\를 / 변경.
    # 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]

    # 파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용을 추가.
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일 내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

아래의 코드를 실행해주세요

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# 입력으로 들어온 token단어들에 대해서 lemmatization 어근 변환.
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# TfidfVectorizer 객체 생성 시 tokenizer인자로 해당 함수를 설정하여 lemmatization 적용
# 입력으로 문장을 받아서 stop words 제거-> 소문자 변환 -> 단어 토큰화 -> lemmatization 어근 변환.
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

2-1 문서를 TF-IDF 형태로 피처 벡터화 하기 위한 코드를 채워주세요.<br>
 tokenizer 인자에는 위에서 실행한 LemNormalize 함수를 , ngram는 (1,2) , min_df는 0.05, max_df는 0.85로 설정해주세요.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = #채워주세요

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

2-2 변환된 피처 벡터화 행렬 데이터에 대해서 군집화를 수행해 어떤 문서끼리 군집되는지 확인하기 위해 K-평균을 적용해주세요.<br>
3개의 중심으로, 최대 반복횟수는 10000으로 random_state=0으로 설정해주세요.

In [ ]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster = #채워주세요
km_cluster.#채워주세요
cluster_label = #채워주세요


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

2-3 opinion text 의 내용 을 확인하여 cluster 0,1,2 가 각각 호텔, 전자기기, 자동차 리뷰 중 어떤 리뷰로 군집화 되었는지 적어주세요.<br>
답:
<br>cluster 0:
<br>cluster 1:
<br>cluster 2:

각 군집에 속한 문서는 핵심 단어를 주축으로 군집화돼 있을 것입니다. 각 군집을 구성하는 핵심단어가 어떤것이 있는지 확인해보겠습니다.

2-4 clusters_centers_의 속성값을 이용해 각 군집별 핵심 단어를 찾아보겠습니다. 아래에 있는 주석을 보고 빈칸에 코드를 채워서 실행해주세요.

In [ ]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}

    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
    centroid_feature_ordered_ind =#채워주세요

    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화.
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num

        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함.
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]

        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values =#채워주세요
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames

    return cluster_details

아래의 코드를 실행해주세요.

In [ ]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')

2-5 위에서 생성한 get_cluster_details(), print_cluster_details() 를 활용하여 아래의 코드를 채워주세요.

In [ ]:
feature_names = tfidf_vect.get_feature_names_out()

cluster_details = get_cluster_details(#채워주세요,
                                      clusters_num=3, top_n_features=10 )
print_cluster_details(cluster_details)

# 문제 3. 문서유사도(50점)

3-(1) 빈칸을 채워주세요. (5점)
- 문서와 문서 간의 유사도 비교는 ____ 를 사용한다.
- 희소행렬 기반에서 ____ 에 기반한 유사도 지표는 정확도가 떨어진다.

3-(2) 코사인 유사도를 구하는 cos_similarity() 함수를 완성해주세요.(10점)

In [1]:
import numpy as np

def cos_similarity(v1, v2):
    dot_product = '''your code'''
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = '''your code'''  
    
    return similarity

3-(3) doc_list의 3개의 문서를 TF-IDF로 백터화된 행렬로 변환해주세요.(10점)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

doc_list = ['if you take the blue pill, the story ends' ,
            'if you take the red pill, you stay in Wonderland',
            'if you take the red pill, I show you how deep the rabbit hole goes']

tfidf_vect_simple = '''your code'''
feature_vect_simple = '''your code'''


3-(4) doc_list의 첫번째 문서와 두번째 문서의 코사인 유사도를 추출해주세요.(10점)

In [ ]:
# (3)의 희소행렬을 Dense Matrix로 변환해주세요.
feature_vect_dense = '''your code'''

#첫번째 문장과 두번째 문장의 feature vector를 추출해주세요.
vect1 = '''your code'''
vect2 = '''your code'''

#구한 feature vector로 코사인 유사도를 출력해주세요.
similarity_simple = '''your code'''
print('문장 1, 문장 2 Cosine 유사도: {0:.3f}'.format(similarity_simple))

3-(5) 첫번째 문장과 세번째 문장, 두번째 문장과 세번째 문장의 코사인 유사도도 추출해주세요.(10점)

In [ ]:
vect1 = '''your code'''
vect3 = '''your code'''
similarity_simple = '''your code'''
print('문장 1, 문장 3 Cosine 유사도: {0:.3f}'.format(similarity_simple))

vect2 = '''your code'''
vect3 = '''your code'''
similarity_simple = '''your code'''
print('문장 2, 문장 3 Cosine 유사도: {0:.3f}'.format(similarity_simple))

3-(6) sklearn.metrics.pairwise의 cosine_similarity를 이용해 doc_list의 모든 쌍의 문서 유사도를 구해주세요.(15점)

In [ ]:
from '''your code''' import '''your code'''

similarity_simple_pair = '''your code'''
print(similarity_simple_pair)

# 문제 4. 한글 텍스트 처리(50점)

4-(1) 빈칸을 채워주세요,(5점)
- 한글 텍스트 처리에서는 영어나 라틴어와 달리 ____ 와 ____ 이슈로 인해 NLP 처리가 까다로운 면이 있다.

KoNLPy를 교재에 제시된 방법으로 먼저 설치한 후에 진행해주세요.

In [ ]:
##아래 코드를 실행시켜주세요
import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df.head(3)

4-(2) 데이터의 기본 가공을 수행해주세요.(10점)

In [ ]:
import re

# 정규 표현식을 이용하여 숫자를 공백으로 변경해주세요.
train_df = train_df.fillna(' ')
train_df['document'] = train_df['document'].'''your code'''

# 테스트 데이터 셋도 Null 및 숫자를 공백으로 변환해주세요.
test_df = pd.read_csv('ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].'''your code'''

4-(3) 한글 형태소 단어로 토큰화하기 위해 tw_tokenizer 함수를 완성시켜주세요.(10점)

In [ ]:
from konlpy.tag import Twitter

twitter = '''your code'''
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환합니다.
    tokens_ko = '''your code'''
    return tokens_ko

4-(4) TF-IDF 피처 모델을 생성해주세요.(시간 오래 걸림 주의)(15점)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
tfidf_vect = '''your code'''

#fit, transform 진행
'''your code'''
tfidf_matrix_train = '''your code'''

4-(5) 로지스틱 회귀 모형을 사용하여 감성분석을 진행해주세요.(10점)

In [ ]:
lg_clf = '''your code'''

# Parameter C 최적화를 위해 GridSearchCV 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV('''your code''' )

#fit
'''your code'''
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

4-(6) 위의 테스트 세트를 이용해 최종 감성 분석 예측을 수행해주세요.(10점)

In [ ]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환 
tfidf_matrix_test = '''your code'''

# GridSearchCV에서 최적 파라미터로 학습된 classifier를 이용
best_estimator = '''your code'''
preds ='''your code'''

print('Logistic Regression 정확도: ','''your code''')
